In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
sox_data = yf.download("^SOX", start="2022-06-29", end="2024-06-27", interval="1d")
print(sox_data.head(20))
print(len(sox_data))

In [ ]:
tsm_data = yf.download("TSM", start="2022-06-29", end="2024-06-27", interval="1d")
print(tsm_data.head(20))
print(len(tsm_data))

In [ ]:
file_path = 'C:/Users/A51607/Desktop/交易競賽/更新的/testData.xlsx'
data = pd.read_excel(file_path)
data['date'] = data['datetime'].dt.date
data['time'] = data['datetime'].dt.time
data['time'] = pd.to_datetime(data['time'], format='%H:%M:%S').dt.time
data.head(30)

In [ ]:
high_low = data['high'] - data['low']
high_close = np.abs(data['high'] - data['close'].shift(1))
low_close = np.abs(data['low'] - data['close'].shift(1))
tr_df = pd.DataFrame({'high_low': high_low, 'high_close': high_close, 'low_close': low_close})
tr = tr_df.max(axis=1)
n = 14
atr_values = tr.rolling(window=n).mean()
data['atr'] = atr_values.fillna(0)
data.head(30)

In [ ]:
data_mor = data[(data['time'] >= pd.to_datetime('08:45:00').time()) & (data['time'] <= pd.to_datetime('13:15:00').time())]
date_list = []
open_list = []
high_list = []
low_list = []
close_list = []
volume_list = []

for i in range(0, len(data_mor), 10):
    chunk = data_mor.iloc[i:i+10]
    date_list.append(chunk['date'].iloc[0])
    open_list.append(chunk['open'].iloc[0])
    high_list.append(chunk['high'].max())
    low_list.append(chunk['low'].min())
    close_list.append(chunk['close'].iloc[-1])
    volume_list.append(chunk['volume'].sum())

daily_data = pd.DataFrame({'Date': date_list, 'Open': open_list, 'High': high_list, 'Low': low_list, 'Close': close_list, 'Volume': volume_list})
daily_data.set_index('Date', inplace=True)
daily_data['Short_EMA'] = daily_data['Close'].ewm(span= 5, adjust=False).mean().shift(1).fillna(0)
daily_data['Long_EMA'] = daily_data['Close'].ewm(span= 20, adjust=False).mean().shift(1).fillna(0)

print(daily_data.tail(20))
print(len(daily_data))

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(sox_data['Close'], label='SOX Index', color='blue')
plt.title('SOX Index')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(tsm_data['Close'], label='TSM ADR', color='orange')
plt.title('TSM ADR')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(daily_data['Close'], label='TX Future', color='red')
plt.title('Taiwan Index Future')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)

In [ ]:
daily_data = daily_data.reset_index()
for i in range(0, len(data_mor)-9, 10):
    for j in range(0, len(daily_data)):
        if data_mor.iloc[i]['date'] == daily_data.loc[j]['Date']:    
            data_mor.loc[i:i+9, 'Short EMA'] = daily_data.loc[j]['Short_EMA']
            data_mor.loc[i:i+9, 'Long EMA'] = daily_data.loc[j]['Long_EMA']

print(data_mor.tail(20))
data_mor.to_excel("C:/Users/A51607/Desktop/交易競賽/更新的/data_mor.xlsx")


In [ ]:
date_series = pd.to_datetime(date_list)
previous_day_series = date_series - pd.Timedelta(days=1)
previous_business_days = [date - pd.offsets.BDay(1) for date in date_series]
previous_business_days_str = [date.strftime('%Y-%m-%d') for date in previous_business_days]
print(previous_business_days_str)

In [ ]:
all_dates = pd.DataFrame(index=pd.to_datetime(previous_business_days_str))
sox_predate = all_dates.join(sox_data, how='left')
sox_predate = sox_predate.rename(columns={'Open': 'S-Open', 'High': 'S-High', 'Low': 'S-Low', 'Close': 'S-Close'})
print(sox_predate)

In [ ]:
f_close = daily_data['Close'].to_list()
s_close = sox_predate['S-Close'].to_list()
merged_df = pd.DataFrame({'Date': date_list, 'F-Close': f_close, 'S-Date': previous_business_days_str, 'S-Close': s_close})
comparison = merged_df.dropna()
print(comparison.head(20))
future_price = pd.Series(comparison['F-Close'])
sox_price = pd.Series(comparison['S-Close'])
correlation = future_price.corr(sox_price)
print(f'SOX & Future Correlation: {correlation}')

In [ ]:
change = yf.download(["^SOX", "TSM"], start="2022-06-29", end="2024-06-27", interval="1d")['Close'].pct_change().shift(1).dropna()
change_df = pd.DataFrame(change)
change_df.reset_index(inplace=True)
change_df.rename(columns={'index': 'Date'}, inplace=True)
change_df.rename(columns={'TSM': 'T-Change'}, inplace=True)
change_df.rename(columns={'^SOX': 'S-Change'}, inplace=True)
change_df

In [ ]:
comparison['Date'] = pd.to_datetime(comparison['Date'])
change_df['Date'] = pd.to_datetime(change_df['Date'])
calculation = pd.merge(comparison, change_df, on='Date', how='left').fillna(0)
calculation.to_excel('C:/Users/A51607/Desktop/交易競賽/更新的/calculation.xlsx')


In [ ]:
data_mor = pd.read_excel("data_mor.xlsx")
calculation = pd.read_excel("calculation.xlsx")
for i in range(0, len(data_mor)-9, 10):
    hit = 0
    for j in range(0, len(calculation)):
        if data_mor.iloc[i]['date'] == calculation.loc[j]['Date']:    
            data_mor.loc[i:i+9, 'S-Change'] = calculation.loc[j]['S-Change']
            data_mor.loc[i:i+9, 'T-Change'] = calculation.loc[j]['T-Change']
            hit = 1
    if hit == 0:
        data_mor.loc[i:i+9, 'S-Change'] = 0
        data_mor.loc[i:i+9, 'T-Change'] = 0
data_mor.to_excel("data_mor.xlsx")

In [ ]:
initial_capital = 100000000
slippage = 0
commission = 50
tax_rate = 0.0002
unit_dollar = 200

In [ ]:
Condition = None
t = 0
stop_profit_point = 400
stop_loss_point = 200
date_list = []
entry_price_list = []

trade_df = pd.DataFrame(columns=['Entry Datetime', 'Original Price', 'Condition', 'Entry Price', 'Amount', 'Exit Datetime', 'Exit Price', 'Trade Profit', 'Net Profit', 'Trade Return',])

for i in range(0, len(data_mor) - 9, 10):
    
    schange = data_mor['S-Change'].iloc[i]
    tchange = data_mor['T-Change'].iloc[i] 
    t = i + 1

    if schange > 0 and data_mor['Short EMA'].iloc[i] > data_mor['Long EMA'].iloc[i]:
        Condition = 'B'
        entry_datetime = data_mor['datetime'].iloc[t]
        original_price = data_mor['open'].iloc[t]
        entry_price = data_mor['open'].iloc[t] + slippage
        amount = ((initial_capital * 0.5) // (entry_price * unit_dollar))
        add_price_threshold = entry_price + data_mor['atr'].iloc[t] * 8
        
        for j in range(t, t + 9):
            
            if data_mor['open'].iloc[j] >= add_price_threshold:
                add_condition = 'B'
                add_entry_datetime = data_mor['datetime'].iloc[j]
                add_original_price = data_mor['open'].iloc[j]
                add_entry_price = data_mor['open'].iloc[j] + slippage
                add_amount = round(amount * 0.5)

                for k in range(j, t + 9):
                
                    if data_mor['high'].iloc[k] >= entry_price + stop_profit_point:
                        add_exit_price = add_entry_price + stop_profit_point
                        add_exit_datetime = data_mor['datetime'].iloc[k]
                        break

                    if data_mor['low'].iloc[k] <= entry_price - stop_loss_point:
                        add_exit_price = entry_price - stop_loss_point
                        add_exit_datetime = data_mor['datetime'].iloc[k]
                        break

                    elif k == t + 8:
                        add_exit_price = data_mor['open'].iloc[k] - slippage
                        add_exit_datetime = data_mor['datetime'].iloc[k]

                add_trade_profit = (add_exit_price - add_entry_price) * unit_dollar * add_amount
                add_net_profit = add_trade_profit - commission - (add_exit_price + add_entry_price) * unit_dollar * tax_rate
                add_trade_return = (add_net_profit / (add_entry_price * unit_dollar * add_amount))
                new_row_add = {'Entry Datetime': add_entry_datetime, 'Original Price': original_price, 
                               'Condition': 'Add', 'Entry Price': add_entry_price, 'Amount': add_amount, 
                               'Exit Datetime': add_exit_datetime, 'Exit Price': add_exit_price, 'Trade Profit': add_trade_profit, 
                               'Net Profit': add_net_profit, 'Trade Return': add_trade_return}
                new_row_add_df = pd.DataFrame([new_row_add])
                trade_df = pd.concat([trade_df, new_row_add_df], ignore_index=True)
                break
                    

            if data_mor['high'].iloc[j] >= entry_price + stop_profit_point:
                exit_price = entry_price + stop_profit_point
                exit_datetime = data_mor['datetime'].iloc[j]
                break

            if data_mor['low'].iloc[j] <= entry_price - stop_loss_point:
                exit_price = entry_price - stop_loss_point
                exit_datetime = data_mor['datetime'].iloc[j]
                break

            elif j == t + 8:
                exit_price = data_mor['open'].iloc[j] - slippage
                exit_datetime = data_mor['datetime'].iloc[j]

        trade_profit = (exit_price - entry_price) * unit_dollar * amount
        net_profit = trade_profit - commission - (exit_price + entry_price) * unit_dollar * tax_rate
        trade_return = (net_profit / (entry_price * unit_dollar * amount))
        new_row = {'Entry Datetime' : entry_datetime, 'Original Price' : original_price, 'Condition' : Condition, 
                   'Entry Price' : entry_price, 'Amount': amount, 'Exit Datetime': exit_datetime, 
                   'Exit Price': exit_price, 'Trade Profit': trade_profit, 'Net Profit': net_profit, 
                   'Trade Return': trade_return}

        
        new_row_df = pd.DataFrame([new_row])
        trade_df = pd.concat([trade_df, new_row_df], ignore_index=True)
        Condition = None
    
    elif schange < 0 and data_mor['open'].iloc[t] < data_mor['open'][i]:
        Condition = 'S'
        entry_datetime = data_mor['datetime'].iloc[t]
        original_price = data_mor['open'].iloc[t]
        entry_price = data_mor['open'].iloc[t] -slippage
        amount = ((initial_capital * 0.05) // (entry_price * unit_dollar))
        
        for j in range(t, t + 9):

            if data_mor['low'].iloc[j] <= entry_price - stop_profit_point:
                exit_price = entry_price - stop_profit_point
                exit_datetime = data_mor['datetime'].iloc[j]
                break

            elif data_mor['high'].iloc[j] >= entry_price + stop_loss_point:
                exit_price = entry_price + stop_loss_point
                exit_datetime = data_mor['datetime'].iloc[j]
                break

            elif j == t + 8:
                exit_price = data_mor['open'].iloc[j] + slippage
                exit_datetime = data_mor['datetime'].iloc[j]

        trade_profit = (entry_price - exit_price) * unit_dollar * amount
        net_profit = trade_profit - commission - (exit_price + entry_price) * unit_dollar * tax_rate
        trade_return = (net_profit / (entry_price * unit_dollar * amount))
        new_row = {'Entry Datetime' : entry_datetime, 'Original Price' :original_price, 'Condition' : Condition, 
                   'Entry Price' : entry_price, 'Amount': amount, 'Exit Datetime': exit_datetime, 
                   'Exit Price': exit_price, 'Trade Profit': trade_profit, 'Net Profit': net_profit, 
                   'Trade Return': trade_return}
        
        new_row_df = pd.DataFrame([new_row])
        trade_df = pd.concat([trade_df, new_row_df], ignore_index=True)
        Condition = None
    
trade_df.to_excel("C:/Users/A51607/Desktop/交易競賽/更新的/trade_record.xlsx")
print(trade_df['Net Profit'].sum())

In [ ]:
trade_df['Cum Profit'] = trade_df['Net Profit'].cumsum()
plt.figure(figsize=(14, 7))
plt.plot(trade_df['Entry Datetime'], trade_df['Cum Profit'], label='Cumulative Net Profit')
plt.xlabel('Date')
plt.ylabel('Cumulative Net Profit (million)')
plt.title('Cumulative Net Profit Over Time')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(trade_df['Entry Datetime'], trade_df['Original Price'], label='Price')
long_trades = trade_df[trade_df['Condition'] == 'B']
plt.scatter(long_trades['Entry Datetime'], long_trades['Original Price'], color='green', marker='^', label='Long Trade', s=25)
short_trades = trade_df[trade_df['Condition'] == 'S']
plt.scatter(short_trades['Entry Datetime'], short_trades['Original Price'], color='red', marker='v', label='Short Trade', s=25)
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Long and Short Trades OVer Time')
plt.legend()
plt.grid(True)
plt.show()